inside the utils section 4 different functions 
--> transcript generation 
--> saving the wav file 
--> split the generated and downloaded tech via llm pass checks 
--> processing the text and fact check using the llm 
--> build the required interface 3 steps --> backend views pages 

In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
video_id = "VGSyGrWWbhA"  # Example video ID for "Never Gonna Give You Up" by Rick Astley

def get_and_save_transcript(video_id, save_path):
    try:
        # Fetch the transcript
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        
        # Convert the list of dicts to a single string
        transcript = " ".join([entry['text'] for entry in transcript_list])
        
        # Save the transcript to a .txt file
        with open(save_path, 'w', encoding='utf-8') as file:
            file.write(transcript)
        
        print(f"Transcript saved to {save_path}")
    except Exception as e:
        print(f"Error fetching or saving transcript: {e}")

# Example usage
 # Example video ID for "Never Gonna Give You Up" by Rick Astley
save_path = "./transcript.txt"  # Path where you want to save the file in your local repository
get_and_save_transcript(video_id, save_path)

Transcript saved to ./transcript.txt


In [2]:
import yt_dlp
import sys

# URL of the YouTube video
url = "https://youtu.be/VGSyGrWWbhA"

# Options for downloading audio
audio_options = {
    'format': 'bestaudio/best',  # Best audio quality
    'postprocessors': [{  # This will convert the downloaded file to mp3
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
    'outtmpl': str(video_id)+'.%(ext)s',  # Output template for filename
    'noplaylist': True,  # Avoid downloading playlist if the URL is from a playlist
}

# Download the audio
with yt_dlp.YoutubeDL(audio_options) as ydl:
    ydl.download([url])
sys.exit(0)

[youtube] Extracting URL: https://youtu.be/VGSyGrWWbhA
[youtube] VGSyGrWWbhA: Downloading webpage
[youtube] VGSyGrWWbhA: Downloading ios player API JSON
[youtube] VGSyGrWWbhA: Downloading mweb player API JSON
[youtube] VGSyGrWWbhA: Downloading m3u8 information
[info] VGSyGrWWbhA: Downloading 1 format(s): 251
[download] Destination: VGSyGrWWbhA.webm
[download] 100% of  136.10MiB in 00:00:33 at 4.05MiB/s     
[ExtractAudio] Destination: VGSyGrWWbhA.wav
Deleting original file VGSyGrWWbhA.webm (pass -k to keep)


SystemExit: 0

/home/mukullight/anaconda3/envs/script/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
video_id = "chunk_0"
num_speakers = 3 #@param {type:"integer"}

language = 'English' #@param ['any', 'English']

model_size = 'large' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'
path = video_id + '.wav'

In [5]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [1]:
!pip install git+https://github.com/speechbrain/speechbrain.git@develop


  Cloning https://github.com/speechbrain/speechbrain.git (to revision develop) to /tmp/pip-req-build-7i_x1z62
  Running command git clone --filter=blob:none --quiet https://github.com/speechbrain/speechbrain.git /tmp/pip-req-build-7i_x1z62
  Resolved https://github.com/speechbrain/speechbrain.git to commit b18646d80c7d5081a99a32f2a3e44cebe2a8249f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!nvidia-smi

Tue Dec 31 11:21:35 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti     Off |   00000000:01:00.0 Off |                  N/A |
| N/A   45C    P5              7W /   35W |     305MiB /   4096MiB |     28%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [3]:
import torch
torch.clear_autocast_cache()

In [ ]:

import whisper
import datetime
import subprocess
import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from pyannote.audio import Audio
from pyannote.core import Segment
import wave
import contextlib
from sklearn.cluster import AgglomerativeClustering
import numpy as np



# Load Whisper model on CPU due to memory constraints
model_size = "base"  
model = whisper.load_model(model_size, device="cuda")

# Load the speaker embedding model
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda")
)

# Function to convert path to .wav if necessary
def ensure_wav(path):
    if path[-3:] != 'wav':
        subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
        return 'audio.wav'
    return path


# Path to your audio file
path = "/home/mukullight/Desktop/transcript_correction/repo/test/chunks/chunk_0.wav"  # Replace with actual path

# Convert to .wav if necessary
path = ensure_wav(path)

try:
    # Transcribe the audio
    result = model.transcribe(path)
    segments = result["segments"]

    # Get audio duration
    with contextlib.closing(wave.open(path, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)

    audio = Audio()

    def segment_embedding(segment):
        start = segment["start"]
        # Whisper might overshoot the end timestamp in the last segment
        end = min(duration, segment["end"])
        clip = Segment(start, end)
        waveform, sample_rate = audio.crop(path, clip)
        
        # Convert waveform to mono by averaging channels
        waveform = waveform.mean(axis=0, keepdims=True) if waveform.shape[0] > 1 else waveform
        return embedding_model(waveform[None])

    # Generate embeddings for each segment
    embeddings = np.zeros(shape=(len(segments), 192))
    for i, segment in enumerate(segments):
        embeddings[i] = segment_embedding(segment)

    # Handle potential NaNs in embeddings
    embeddings = np.nan_to_num(embeddings)

    # Number of speakers to cluster into (you might want to adjust this dynamically)
    num_speakers = 2  # Example, adjust based on your audio content
    clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
    labels = clustering.labels_

    # Assign speaker labels to segments
    for i in range(len(segments)):
        segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

    # Helper function for formatting time
    def format_time(secs):
        return datetime.timedelta(seconds=round(secs))

    # Write transcript
    with open("transcript.txt", "w") as f:
        for i, segment in enumerate(segments):
            if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
                f.write("\n" + segment["speaker"] + ' ' + str(format_time(segment["start"])) + '\n')
            f.write(segment["text"][1:] + ' ')

    print("Transcription completed and saved to transcript.txt")

except Exception as e:
    print(f"An error occurred: {e}")

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [4]:
import torch
torch.cuda.empty_cache()

In [1]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)  # For the first GPU

# Get memory info
info = pynvml.nvmlDeviceGetMemoryInfo(handle)
print("Memory Total: {0} MiB".format(info.total / 1024 / 1024))
print("Memory Used: {0} MiB".format(info.used / 1024 / 1024))
print("Memory Free: {0} MiB".format(info.free / 1024 / 1024))

# Get processes using this GPU
procs = pynvml.nvmlDeviceGetComputeRunningProcesses(handle)
for p in procs:
    print("Process ID: {0}, Memory usage: {1} MiB".format(p.pid, p.usedGpuMemory / 1024 / 1024))

pynvml.nvmlShutdown()

Memory Total: 4096.0 MiB
Memory Used: 830.5 MiB
Memory Free: 3265.5 MiB


In [7]:
!pip install pyannote.audio

In [2]:
!nvidia-smi

Tue Dec 31 09:39:55 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti     Off |   00000000:01:00.0 Off |                  N/A |
| N/A   46C    P5              7W /   50W |     328MiB /   4096MiB |     38%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

class SpeakerNumberEstimator:
    def __init__(self, embedding_extractor):
        self.embedding_extractor = embedding_extractor  # Assume this is your model for extracting speaker embeddings

    def estimate_number_of_speakers(self, audio_data, max_speakers=10):
        # Extract embeddings
        embeddings = self.embedding_extractor(audio_data).detach().numpy()
        
        # Silhouette Method for determining number of clusters
        silhouette_scores = []
        for k in range(1, max_speakers + 1):  # Start from 2 as at least two speakers needed for separation
            kmeans = KMeans(n_clusters=k, random_state=0).fit(embeddings)
            score = silhouette_score(embeddings, kmeans.labels_)
            silhouette_scores.append(score)
        
        # Find the peak silhouette score which indicates the optimal number of clusters
        optimal_k = np.argmax(silhouette_scores) + 1 # +2 because we started counting from 2
        
        return optimal_k

# Hypothetical usage:
# Assume 'model' is your pre-trained embedding extractor
estimator = SpeakerNumberEstimator(model)
audio = torch.randn(1, 16000)  # Example audio tensor
num_speakers = estimator.estimate_number_of_speakers(audio)
print(f"Estimated number of speakers: {num_speakers}")

In [4]:
# enable memory history, which will
# add tracebacks and event history to snapshots

torch.cuda.memory._dump_snapshot("my_snapshot.pickle")

In [1]:
import wave
import numpy as np
import os

def split_wav(wav_file, chunk_size_ms, output_dir='./chunks'):
    """
    Split a WAV file into chunks of specified duration.

    :param wav_file: Path to the input WAV file
    :param chunk_size_ms: Size of each chunk in milliseconds (default 30 seconds)
    :param output_dir: Directory to save the chunks
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open the WAV file
    with wave.open(wav_file, 'rb') as wf:
        # Read parameters
        params = wf.getparams()
        nchannels, sampwidth, framerate, nframes = params[:4]
        
        # Read all frames
        frames = wf.readframes(nframes)
        
        # Convert frames to numpy array for easier manipulation
        audio_data = np.frombuffer(frames, dtype=np.int16)
        
        # Calculate number of frames per chunk
        chunk_frames = int(framerate * chunk_size_ms / 1000)
        
        # Split the audio data into chunks
        for i in range(0, len(audio_data), chunk_frames):
            chunk = audio_data[i:i + chunk_frames]
            
            # If this is not the last chunk and it's shorter than chunk_frames, pad it
            if i + chunk_frames > len(audio_data) and len(chunk) < chunk_frames:
                padding = np.zeros(chunk_frames - len(chunk), dtype=np.int16)
                chunk = np.concatenate([chunk, padding])
            
            # Convert back to bytes
            chunk_bytes = chunk.astype(np.int16).tobytes()
            
            # Create new WAV file for each chunk
            chunk_filename = os.path.join(output_dir, f'chunk_{i // chunk_frames}.wav')
            with wave.open(chunk_filename, 'wb') as out:
                out.setparams(params)
                out.writeframes(chunk_bytes)

# Example usage
wav_file_path = '/home/mukullight/Desktop/transcript_correction/repo/test/VGSyGrWWbhA.wav'
split_wav(wav_file_path, chunk_size_ms=300000)  

In [3]:
pip install nemo

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 277.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 1.2 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 4.1 MB/s eta 0:00:0000:0100:01
  Created wheel for drf-flex-fields: filename=drf_flex_fields-1.0.2-py3-none-any.whl size=16641 sha256=e09010fe7c8e893adf4baeee7b8292f79a8ecbfe0f8c1413ecdce81b28834088
  Stored in directory: /home/mukullight/.cache/pip/wheels/15/9e/6c/95a7cd990d4561e61f3dd0b8408e33d0de3de70c2c1ebfb40b
Successfully built drf-flex-fields
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.2
    Uninstalling pytz-2024.2:
      Successfully uninstalled pytz-2024.2
  Attempting uninstall: python-dateutil
    Found existing i

In [1]:
import nemo.collections.asr as nemo_asr
diarizer_model = nemo_asr.models.EncDecDiarLabelModel.from_pretrained(model_name="diarizer")

ModuleNotFoundError: No module named 'nemo'

In [ ]:
from omegaconf import OmegaConf
cfg = OmegaConf.create({
    'window_length_in_sec': 1.5,
    'shift_length_in_sec': 0.75,
    'multiscale_weights': [1.0, 0.5],
    'max_num_speakers': 8
})
diarizer_model.setup_diarizer(cfg)

In [ ]:
diarizer_model.diarize(audio_file_path="path/to/your/audio.wav", out_file="output.rttm")